In [1]:
# install
using Pkg

Pkg.add("NLsolve")
Pkg.add("Optim")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed Parameters ─────────── v0.12.0
 Installed LineSearches ───────── v7.0.1
 Installed NLsolve ────────────── v4.2.0
 Installed SpecialFunctions ───── v0.9.0
 Installed OrderedCollections ─── v1.1.0
 Installed NLSolversBase ──────── v7.5.0
 Installed ForwardDiff ────────── v0.10.7
 Installed OpenSpecFun_jll ────── v0.5.3+1
 Installed NaNMath ────────────── v0.3.3
 Installed DiffEqDiffTools ────── v1.5.0
 Installed DiffRules ──────────── v0.1.0
 Installed Calculus ───────────── v0.5.1
 Installed ArrayInterface ─────── v2.0.0
 Installed Distances ──────────── v0.8.2
 Installed Requires ───────────── v0.5.2
 Installed DiffResults ────────── v0.0.4
 Installed Reexport ───────────── v0.2.0
 Installed StaticArrays ───────── v0.12.1
 Installed CommonSubexpressions ─ v0.2.0
  Updating `~/.julia/environments/v1.3/Project.toml`
  [2774e3e

In [2]:
using NLsolve

┌ Info: Precompiling NLsolve [2774e3e8-f4cf-5e23-947b-6d7e65073b56]
└ @ Base loading.jl:1273


A graph is usualy given by the list of links [(source, target), ...]. However, for the particular type of graph, chains of nodes are considered. So the graph is given by chains of nodes: [[A1, A2, ...], ...].

In [2]:
graph = [["074559265", "033293546", "177194634", "132327031", "05942608X", "147936594",
        "194946746", "20308523X", "160289750", "103659625", "103664262", "07456501X",
        "074559192", "074576127", "113270321", "123899249", "122925262", "121644545",
        "122935314", "13235103X", "149658192", "149531125", "151099103", "168078465",
        "17817310X", "18105938X", "181232529", "195092317", "202363856"],
    ["069862753", "033922020", "185417396", "185923089", "161170161", "188021574",
        "188160396", "188705945", "189504048", "189565624", "189997680", "190237201",
        "06986263X", "074589644", "076964361", "076694348", "076969576", "083593217",
        "110189159", "129545279"], ["112512380", "109209265", "185124100", "180818260",
        "185775926", "185887015", "187849390", "18786425X", "188021574", "112508901"],
    ["129545678", "129545279", "140093052", "154959588", "155289756", "176736999",
        "177538619", "179791338", "177523824", "195092317"],
    ["177539135", "177538619", "200874586", "241318939"],
    ["074576127", "168078465", "181232529"], ["188160396", "236926446"]];

In [67]:
graph = [["one", "two", "three"],
         ["two", "four", "five"]];

In [68]:
nodes = Dict(n=>rand(Float32, 2) for ch in graph for n in ch);
println("#nodes: ", length(nodes))
println("#chains: ", length(graph))

#nodes: 5
#chains: 2


Next we need a function which compute the system energy (and the forces?) for a given graph and a list of nodes coordinates. Folowing classical approach [Fruchterman and Reingold 1991](http://www.mathe2.uni-bayreuth.de/axel/papers/reingold:graph_drawing_by_force_directed_placement.pdf) there is an attractive force -between linked nodes- and repulsive forces between every nodes. In addition, here we want a torque at each node in a chain to rigidify the line. Maybe, a global gravity field would be required to densify the graph (keep disconected parts). 

$$
f_{attractive} = d^2/k \\
f_{repulsive} = -k^2/d
$$

In [69]:
norm(x) = sqrt(sum(x.^2));

In [70]:
const k = .1 # whatever the unit

function attractive_energy(x1, x2)
    d2 = norm(x1 .- x2).^2
    E = d2/k/3
    end;

function repulsive_energy(x1, x2)
    d = norm(x1 .- x2)
    E = -k^2*log(d)
    end;

#println( attractive_energy(nodes["033293546"], nodes["177194634"]) )
#println( repulsive_energy(nodes["033293546"], nodes["177194634"]) )

test http://www.juliadiff.org/ForwardDiff.jl/stable/

In [71]:
using(ForwardDiff)

The torque will be function of the angle formed between 3 ordered nodes. 

In [34]:
coords = [[3. 3.]; [1. 5.]]

2×2 Array{Float64,2}:
 3.0  3.0
 1.0  5.0

In [36]:
coords[1, :]

2-element Array{Float64,1}:
 3.0
 3.0

In [51]:
function distance(x)
    d = sum((x[2, :] .- x[1, :]).^2)
    end;

function square(x)
    d = sum(x.^2)
    end;

function sum_distance(x)
    d = 0
    for i in size(x)[1]-1
        d += sum((x[i+1, :] .- x[i, :]).^2)
        end;
    return d
    end;


In [72]:
coords = [[3. 3.]; [1. 5.]; [5.4 12.]; [1. 3.]]
sum_distance(coords)

100.36

In [60]:
size(coords)

(4, 2)

In [61]:
distance(coords)

8.0

In [62]:
ForwardDiff.gradient(sum_distance, coords)

4×2 Array{Float64,2}:
 -0.0   -0.0
 -0.0   -0.0
  8.8   18.0
 -8.8  -18.0

In [73]:
function join_angle(x1, x2, x3)
    x12 = x2 .- x1
    x23 = x3 .- x2
    dot = sum( x12 .* x23 )/(norm(x12)*norm(x23))
    theta = acos(dot)
    end;

function join_energy(x1, x2, x3)
    theta = join_angle(x1, x2, x3)
    gamma = .008 # J/rad2
    E = gamma*theta^2
    end;

#x1, x2, x3 = nodes["033293546"], nodes["177194634"], nodes["132327031"]
#println(join_angle(x1, x2, x3))
#println(join_energy(x1, x2, x3))

In [74]:
function isolated_chain_energy(chain, nodes)
    nrj = 0
    for k in 1:length(chain)-2
        n1, n2, n3 = chain[k], chain[k+1], chain[k+2]
        x1, x2, x3 = nodes[n1], nodes[n2], nodes[n3]
        nrj += join_energy(x1, x2, x3)
        nrj += attractive_energy(x1, x2)
        end;
    # last link:
    nrj += attractive_energy(nodes[chain[end-1]], nodes[chain[end]])
    end;

function global_chain_energy(graph, nodes)
    nrj = 0
    for chain in graph
        nrj += isolated_chain_energy(chain, nodes)
        end;
    return nrj
    end;

In [75]:
chain = graph[2]
isolated_chain_energy(chain, nodes)

1.2304188557068507

In [76]:
global_chain_energy(graph, nodes)

3.494590395748615

In [77]:
function global_repulsive_energy(nodes)
    nrj = 0
    for (n1, x1) in pairs(nodes)
        for (n2, x2) in pairs(nodes)
            if n1 > n2
                nrj += repulsive_energy(x1, x2)
                end;
            end;
        end;
    return nrj
    end;

println(global_repulsive_energy(nodes))

function graph_energy(graph, nodes)
    nrj = global_repulsive_energy(nodes)
    nrj += isolated_chain_energy(chain, nodes)
    return nrj
    end;

0.08119494974613192


In [78]:
graph_energy(graph, nodes)

1.3116138054529827

Let's now try to solve this mess

In [84]:
nodes_list = String[]
coords = Float32[]
for (n, x) in pairs(nodes)
    push!(nodes_list, n)
    append!(coords, x)
    end;

In [80]:
function proxy_function(x)
    for (i, n) in enumerate(nodes_list)
        nodes[n] = x[i, :]
        end;
    E = graph_energy(graph, nodes)
    end;

In [93]:
coords[1, :]

2-element Array{Float64,1}:
 3.0
 3.0

In [89]:
coords = [[3. 3.]; [1. 5.]; [5.4 12.]; [1.5 3.1]; [5. 2.]; [5. 1.]]

6×2 Array{Float64,2}:
 3.0   3.0
 1.0   5.0
 5.4  12.0
 1.5   3.1
 5.0   2.0
 5.0   1.0

In [90]:
proxy_function(coords)

109.92440395589668

In [91]:
coords

6×2 Array{Float64,2}:
 3.0   3.0
 1.0   5.0
 5.4  12.0
 1.5   3.1
 5.0   2.0
 5.0   1.0

In [92]:
ForwardDiff.gradient(proxy_function, coords)

MethodError: MethodError: no method matching Float32(::ForwardDiff.Dual{ForwardDiff.Tag{typeof(proxy_function),Float64},Float64,12})
Closest candidates are:
  Float32(::Real, !Matched::RoundingMode) where T<:AbstractFloat at rounding.jl:200
  Float32(::T) where T<:Number at boot.jl:718
  Float32(!Matched::Int8) at float.jl:60
  ...

In [61]:
proxy_function(coords)

0.5143664758205415

In [39]:
nlsolve(c, coords, iterations=60)

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: Float32[0.5859971, 0.65081704, 0.43273032, 0.5519515, 0.5954083, 0.6544633, 0.6554451, 0.57317173, 0.5526732, 0.66727567]
 * Zero: Float32[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]
 * Inf-norm of residuals: 0.346776
 * Iterations: 60
 * Convergence: false
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: false
 * Function Calls (f): 61
 * Jacobian Calls (df/dx): 1

https://github.com/JuliaNLSolvers/Optim.jl

In [63]:
proxy_function(coords)

0.5143664758205415

In [64]:
result = Optim.optimize(proxy_function, coords)

 * Status: failure (reached maximum number of iterations) (line search failed)

 * Candidate solution
    Minimizer: [1.14e+04, 1.54e+04, 1.14e+04,  ...]
    Minimum:   -8.167206e-01

 * Found with
    Algorithm:     Nelder-Mead
    Initial Point: [5.86e-01, 6.51e-01, 4.33e-01,  ...]

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    1457


In [67]:
result.minimizer

10-element Array{Float32,1}:
   11352.758
   15428.459
   11352.742
   15428.418
   15338.558
  348802.9  
  122413.36 
 -585183.7  
   11352.746
   15428.359

In [43]:
using(Optim)

┌ Info: Precompiling Optim [429524aa-4258-5aef-a3af-852621145aeb]
└ @ Base loading.jl:1273
